In [ ]:
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, BigInteger, Sequence, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import os
from libv3.utils import *
import pandas as pd
import json
import shutil

# Define the base for the declarative model
Base = declarative_base()

# Define the Event class which will be mapped to the events table in the database
class Event(Base):
    __tablename__ = 'events'
    
    id: Mapped[int] = mapped_column(primary_key=True)
    
    name: Mapped[str] = mapped_column(String(50), nullable=False)
    timestamp: Mapped[int] = mapped_column(BigInteger, nullable=False)
    file_number: Mapped[int] = mapped_column(ForeignKey("file_config.id")) 

    config: Mapped["File_config"] = relationship(back_populates="events")
    


class File_config(Base):
    __tablename__ = 'file_config'

    id: Mapped[int] = mapped_column(primary_key=True)
    code_base: Mapped[str] = mapped_column(String(50), nullable=False)
    version: Mapped[int] = mapped_column(Integer, nullable=False)
    behaviour: Mapped[str] = mapped_column(String(50), nullable=False)
    trial_num: Mapped[int] = mapped_column(Integer, nullable=False)

    events: Mapped[List["Event"]] = relationship(back_populates="config")

# # Function to add events to the database
# def add_events(event_list):
#     for event_name, event_timestamp in event_list:
#         event = Event(name=event_name, timestamp=event_timestamp)
#         session.add(event)
#     session.commit()





In [ ]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config('wolfnet_config')   ### config stored in libv3/exp_config.txt
TRIAL = 1

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/trial{TRIAL}'

print(log_path)

#### file to display
trace_file = 14

print('file number:', trace_file)

In [ ]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

### remove.Ds_store from all lists
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

### Combine files from trials

In [ ]:
##### combine files
paths_traces.sort(key=lambda x: int(x.split('/')[-1][5:]))
last_file_num = int(paths_traces[-1].split('/')[-1][5:])
all_content = []
for trace_path in paths_traces:
    content = read_traces(trace_path)   
    all_content.extend(content)
    # print(content)

# path_to_save = os.path.dirname(os.path.dirname(trace_path))
# with open(path_to_save+f'/trace_trial{TRIAL}', 'w') as f:
#     json.dump(all_content, f)
#     # print('Saved trace:', path_to_save+f'/trace_trial{TRIAL})

# varlist = read_json(os.path.join(os.path.dirname(trace_path),f'varlist{last_file_num}'))
# # print(varlist)
# with open(path_to_save+f'/varlist_trial{TRIAL}.json', 'w') as f:
#     json.dump(varlist, f)



In [ ]:
path_to_content = os.path.dirname(log_path)+f'/trace_trial{TRIAL}'
content_towrite = read_traces(path_to_content)
# print(content_towrite)
# Create an SQLite database (or connect to it if it already exists)
database_url = 'sqlite:///events.db'
engine = create_engine(database_url, echo=True)

# Create all tables in the database
Base.metadata.create_all(engine)

# # Create a configured "Session" class
Session = sessionmaker(bind=engine)

new_log_instance = File_config(code_base=CODE, version=VER, behaviour=BEHAVIOUR, trial_num=TRIAL)
print(new_log_instance.id)
# # #### since we use session.begin(), it will commit and close the session automatically as the with block ends
with Session.begin() as session:
    session.add(new_log_instance)
    session.commit()

with Session.begin() as session:
    for event_name, event_timestamp in all_content:
        event = Event(name=event_name, timestamp=event_timestamp, config=new_log_instance)
        session.add(event)
        # break



In [ ]:
#### query the database

with Session() as session:
    results = session.query(File_config).where(File_config.id == 1).all()
    print(results[0].id)
    print(results[0].code_base)
    print(results[0].version)
    print(results[0].behaviour)
    print(results[0].trial_num)
    print(results[0].events)